# 0. Import Estimator API Module

In [1]:
import sys
sys.path.append('../../')

In [2]:
from aistudio.estimator3 import Estimator

In [3]:
print(Estimator)

<class 'aistudio.estimator3.Estimator'>


In [4]:
print(sys.path)

['/home/sky/dev/aistudio/workspace/ws-1', '/home/sky/anaconda3/envs/ai/lib/python37.zip', '/home/sky/anaconda3/envs/ai/lib/python3.7', '/home/sky/anaconda3/envs/ai/lib/python3.7/lib-dynload', '', '/home/sky/anaconda3/envs/ai/lib/python3.7/site-packages', '/home/sky/tmp/jupyterlab-git', '/home/sky/anaconda3/envs/ai/lib/python3.7/site-packages/IPython/extensions', '/home/sky/.ipython', '../../']


In [5]:
import inspect
print(inspect.getfile(Estimator))

../../aistudio/estimator3.py


# 1. Prepare Dataset

In [ ]:
import torch
from torchvision import datasets, transforms

download_root = 'train-data'
mnist_transform=transforms.Compose([
                           transforms.ToTensor()
                           ,transforms.Normalize((0.1307,), (0.3081,))
                           ])
train_dataset  = datasets.MNIST(download_root, transform=mnist_transform, train=True, download=True)
input_data_torch = train_dataset.data
input_data_torch = torch.div(input_data_torch,255.)
input_data_torch = torch.add(input_data_torch,-0.1307)
input_data_torch = torch.div(input_data_torch,0.3081)
input_data_torch = input_data_torch.unsqueeze(1)
input_labels_torch = train_dataset.targets
# Convert Tensor to Numpy
input_data = input_data_torch.numpy()
input_labels = input_labels_torch.numpy()

<div class="alert alert-block alert-info">
    Check data type and shape
</div>

In [ ]:
print(type(input_data))
print(input_data.shape)

<div class="alert alertbox alert-info">
    Set Script Parameters
</div>

In [ ]:
script_params = {
    'epochs':5,
    'batch-size':64,
    'test-batch-size':128,
    'lr':0.01,
    'momentum':0.5,
    'seed':42,
    'log-interval':10,
    #'no-cuda': True, # True is dummy value, value is always True
    'nprocs':1,
    #'loss':'cross_entropy',
    'loss':'nll_loss',
    'optimizer':'SGD',
    'debug': True # True is dummy value, value is always True
}

## Scenario 1 (with model name in the aistudio service)

In [ ]:
estimator = Estimator(model_name="model-1",script_params=script_params)

In [ ]:
estimator.fit(input_data,input_labels)

In [ ]:
# %%writefile './estimator3_test.py'

# from aistudio.estimator3 import Estimator
# import torch
# from torchvision import datasets, transforms

# download_root = 'train-data'
# mnist_transform=transforms.Compose([
#                            transforms.ToTensor()
#                            ,transforms.Normalize((0.1307,), (0.3081,))
#                            ])
# train_dataset  = datasets.MNIST(download_root, transform=mnist_transform, train=True, download=True)
# input_data_torch = train_dataset.data
# input_data_torch = torch.div(input_data_torch,255.)
# input_data_torch = torch.add(input_data_torch,-0.1307)
# input_data_torch = torch.div(input_data_torch,0.3081)
# input_data_torch = input_data_torch.unsqueeze(1)
# input_labels_torch = train_dataset.targets
# # Convert Tensor to Numpy
# input_data = input_data_torch.numpy()
# input_labels = input_labels_torch.numpy()
# script_params = {
#     'epochs':5,
#     'batch-size':64,
#     'test-batch-size':128,
#     'lr':0.01,
#     'momentum':0.5,
#     'seed':42,
#     'log-interval':10,
#     'no-cuda':False,
#     'nprocs':1,
#     'loss':'cross_entropy',
#     'optimizer':'SGD'
# }
# estimator = Estimator(model_name="model-1",script_params=script_params)
# estimator.fit(input_data,input_labels)

# Scenario 2 (with model object)

## Let's make network and load model file

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

In [ ]:
model = Net()
model.load_state_dict(torch.load("./mnist_net.pth"))

In [ ]:
estimator = Estimator(model_name="model-2",script_params=script_params)

In [ ]:
estimator.fit(input_data,input_labels)

# New Scenario

In [ ]:
# USER-DEFINED - Feel free to change
modulename = "torchnet"
######### DO NOT CHANGE #########
filename = modulename + ".py"
filename = "./nets/" + filename
#################################

<div class="alert alert-block alert-warning">
Do not change code below at Line 1 to 3
</div>

In [ ]:
%%writefile $filename
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

In [ ]:
script_params = {
    'epochs':5,
    'batch-size':64,
    'test-batch-size':128,
    'lr':0.01,
    'momentum':0.5,
    'seed':42,
    'log-interval':10,
    'no-cuda':False,
    'nprocs':1,
    #'loss':'cross_entropy',
    'loss':'nll_loss',
    'optimizer':'SGD',
    'debug': True
}

In [ ]:
estimator = Estimator(net_name=modulename,script_params=script_params)

In [ ]:
estimator.fit(input_data,input_labels)

# New Scenario2

<div class="alert alert-block alert-info">
<h1>Example 1(MNIST-deeplearning.net)</h1>
</div>

<div class="alert alert-block alert-info">
Feel free to change modulename
</div>

In [ ]:
# USER-DEFINED
modulename = "mnist-linear"
######### DO NOT CHANGE #########
net_filename = modulename + ".py"
net_filename = "./nets/" + net_filename
#################################

<div class="alert alert-block alert-warning">
    <p>Do not change code below at Line 1 to 3 </p>
    <p>Do not change class name <b>'Net'</b></p>
</div>

In [ ]:
%%writefile $net_filename
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)

In [ ]:
# USER-DEFINED - Feel free to change
dataset_name = "MNIST"
######### DO NOT CHANGE #########
dataset_filename = dataset_name + ".py"
dataset_filename = "./datasets/" + dataset_filename
#################################

In [ ]:
%%writefile $dataset_filename

from pathlib import Path
import requests
import pickle
import gzip
import numpy as np
from torch.utils.data import TensorDataset
import torch
import os

class DatasetLoader:
    
    def __init__(self):
        ########## WRITE DATASET LOADER CODE HERE ##########
        
        DATA_PATH = Path("datasets")
        PATH = DATA_PATH / "mnist"

        PATH.mkdir(parents=True, exist_ok=True)

        URL = "http://deeplearning.net/data/mnist/"
        FILENAME = "mnist.pkl.gz"

        if not (PATH / FILENAME).exists():
                content = requests.get(URL + FILENAME).content
                (PATH / FILENAME).open("wb").write(content)

        with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
                ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")       

        x_train, y_train, x_valid, y_valid = map(
            torch.tensor, (x_train, y_train, x_valid, y_valid)
        )
        ### train_ds and valid_ds MUST BE TensorDataset(or ImageFolder or Torch Dataset Format)
        self.train_dataset = TensorDataset(x_train, y_train)
        self.valid_dataset = TensorDataset(x_valid, y_valid)
        ####################################################
        
    def get_train_dataset(self, validation=True):        
        if validation is True:
            return self.train_dataset, self.valid_dataset
        else:
            return self.train_dataset
    
    def get_test_dataset(self):
        return self.test_dataset

In [ ]:
script_params = {
    'epochs':5,
    'batch-size':64,
    'test-batch-size':128,
    'lr':0.01,
    'momentum':0.5,
    'seed':42,
    'log-interval':10,
    'no-cuda':False,
    'nprocs':1,
    'loss':'cross_entropy',
    #'loss':'nll_loss',
    'optimizer':'SGD',
    'debug': True
}

In [ ]:
estimator = Estimator(net_name=modulename,script_params=script_params)

In [ ]:
estimator.fit(dataset_loader=dataset_name)

<div class="alert alert-block alert-info">
<h1>Example 2(MNIST-torchvision)</h1>
</div>

<div class="alert alert-block alert-info">
Feel free to change modulename
</div>

In [6]:
# USER-DEFINED
modulename = "mnist-CNN"
######### DO NOT CHANGE #########
net_filename = modulename + ".py"
net_filename = "./nets/" + net_filename
#################################

<div class="alert alert-block alert-warning">
    <p>Do not change code below at Line 1 to 3 </p>
    <p>Do not change class name <b>'Net'</b></p>
</div>

In [7]:
%%writefile $net_filename
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

Overwriting ./nets/mnist-CNN.py


<div class="alert alert-block alert-info">
Feel free to change dataset name
</div>

In [8]:
# USER-DEFINED
dataset_name = "MNIST2"
######### DO NOT CHANGE #########
dataset_filename = dataset_name + ".py"
dataset_filename = "./datasets/" + dataset_filename
#################################

In [9]:
%%writefile $dataset_filename

from torchvision import datasets, transforms
import os

class DatasetLoader:    
    def __init__(self):
        ########## WRITE DATASET LOADER CODE HERE ##########
        thispath = os.path.dirname(os.path.abspath(__file__))
        data_dir = os.path.join(thispath,"MNIST2")
        mnist_transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))])
        train_dataset = datasets.MNIST(data_dir, transform=mnist_transform, train=True,  download=True)        
        test_dataset  = datasets.MNIST(data_dir, transform=mnist_transform, train=False, download=True)
        
        ### train_ds and valid_ds MUST BE TensorDataset(or ImageFolder or Torch Dataset Format)
        self.train_dataset = train_dataset
        self.valid_dataset = test_dataset
        self.test_dataset = test_dataset
        ####################################################
    
    def get_train_dataset(self, validation=True):        
        if validation is True:
            return self.train_dataset, self.valid_dataset
        else:
            return self.train_dataset
    
    def get_test_dataset(self):
        return self.test_dataset

Overwriting ./datasets/MNIST2.py


In [10]:
script_params = {
    'epochs':5,
    'batch-size':64,
    'test-batch-size':128,
    'lr':0.01,
    'momentum':0.5,
    'seed':42,
    'log-interval':10,
    #'no-cuda':False,
    'nprocs':1,
    #'loss':'cross_entropy',
    'loss':'nll_loss',
    'optimizer':'SGD',
    'validation': True,
    'debug': True    
}

In [11]:
estimator = Estimator(net_name=modulename,script_params=script_params)

['--epochs', '5', '--batch-size', '64', '--test-batch-size', '128', '--lr', '0.01', '--momentum', '0.5', '--seed', '42', '--log-interval', '10', '--nprocs', '1', '--loss', 'nll_loss', '--optimizer', 'SGD', '--validation', '--debug', '--net-name', 'mnist-CNN']


In [16]:
estimator.fit(dataset_loader=dataset_name)

Job Requested to Portal.
[1,0]<stdout>:Namespace(batch_size=64, dataset_loader='MNIST2', debug=True, epochs=5, log_interval=10, loss='nll_loss', lr=0.01, model_path=None, momentum=0.5, net_name='mnist-CNN', no_cuda=False, nprocs=1, optimizer='SGD', seed=42, test_batch_size=128, use_adasum=False, validation=True)
[1,0]<stdout>:Arguments Parsing Finished.
[1,0]<stdout>:CUDA Supported!
[1,0]<stdout>:Network was found.
[1,0]<stdout>:Model's state_dict:
[1,0]<stdout>:conv1.weight 	 torch.Size([10, 1, 5, 5])
[1,0]<stdout>:conv1.bias 	 torch.Size([10])
[1,0]<stdout>:conv2.weight 	 torch.Size([20, 10, 5, 5])
[1,0]<stdout>:conv2.bias 	 torch.Size([20])
[1,0]<stdout>:fc1.weight 	 torch.Size([50, 320])
[1,0]<stdout>:fc1.bias 	 torch.Size([50])
[1,0]<stdout>:fc2.weight 	 torch.Size([10, 50])
[1,0]<stdout>:fc2.bias 	 torch.Size([10])
[1,0]<stdout>:Optimizer's state_dict:
[1,0]<stdout>:state 	 {}
[1,0]<stdout>:param_groups 	 [{'lr': 0.009999999776482582, 'momentum': 0.5, 'dampening': 0, 'weight_deca

### predict by the model

### register model

In [14]:
estimator.register_model(model_name="mnist-good-model")

<div class="alert alert-block alert-info">
<h1>Example 3(CIFAR10)</h1>
</div>

In [ ]:
# USER-DEFINED - Feel free to change
modulename = "CIFAR10-CNN"
######### DO NOT CHANGE #########
net_filename = modulename + ".py"
net_filename = "./nets/" + net_filename
#################################

<div class="alert alert-block alert-warning">
    <p>Do not change code below at Line 1 to 3 </p>
    <p>Do not change class name <b>'Net'</b></p>
</div>

In [ ]:
%%writefile $net_filename
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

<div class="alert alert-block alert-info">
Feel free to change dataset name
</div>

In [ ]:
# USER-DEFINED
dataset_name = "CIFAR10"
######### DO NOT CHANGE #########
dataset_filename = dataset_name + ".py"
dataset_filename = "./datasets/" + dataset_filename
#################################

In [ ]:
%%writefile $dataset_filename

from __future__ import print_function, division

import torch
import torchvision
from torchvision import datasets, models, transforms
import os

class DatasetLoader:    
    def __init__(self):
        ########## WRITE DATASET LOADER CODE HERE ##########
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        thispath = os.path.dirname(os.path.abspath(__file__))
        data_dir = os.path.join(thispath,"CIFAR10")
        trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True,
                                                download=True, transform=transform)
        testset = torchvision.datasets.CIFAR10(root=data_dir, train=False,
                                               download=True, transform=transform)
        self.train_dataset = trainset
        self.valid_dataset = testset
        self.test_dataset = testset
        ####################################################
        
    def get_train_dataset(self, validation=True):        
        if validation is True:
            return self.train_dataset, self.valid_dataset
        else:
            return self.train_dataset
    
    def get_test_dataset(self):
        return self.test_dataset

In [ ]:
script_params = {
    'epochs':5,
    'batch-size':32,
    'test-batch-size':64,
    'lr':0.01,
    'momentum':0.9,
    'seed':42,
    'log-interval':10,
    #'no-cuda':False,
    'nprocs':1,
    'loss':'cross_entropy',
    #'loss':'nll_loss',
    'optimizer':'SGD',
    'debug': True
}

In [ ]:
estimator = Estimator(net_name=modulename,script_params=script_params)

In [ ]:
estimator.fit(dataset_loader=dataset_name)

<div class="alert alert-block alert-info">
<h1>Example 4(hymenoptera)</h1>
</div>

In [ ]:
# USER-DEFINED - Feel free to change
modulename = "resnet18"
######### DO NOT CHANGE #########
net_filename = modulename + ".py"
net_filename = "./nets/" + net_filename
#################################

<div class="alert alert-block alert-warning">
    <p>Do not change code below at Line 1 to 3 </p>
    <p>Do not change class name <b>'Net'</b></p>
</div>

In [ ]:
%%writefile $net_filename
from torchvision import models
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):        
        super(Net, self).__init__()        
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        # Here the size of each output sample is set to 2.
        # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
        self.model.fc = nn.Linear(num_ftrs, 2)        
        #self.add_module("resnet18", self.model)
    
    def forward(self, x):
        x = self.model(x)
        return x

<div class="alert alert-block alert-info">
Feel free to change dataset name
</div>

In [ ]:
# USER-DEFINED
dataset_name = "hymenoptera"
######### DO NOT CHANGE #########
dataset_filename = dataset_name + ".py"
dataset_filename = "./datasets/" + dataset_filename
#################################

In [ ]:
%%writefile $dataset_filename

from __future__ import print_function, division

import torch
import torchvision
from torchvision import datasets, models, transforms
import os

class DatasetLoader:    
    def __init__(self):
        ########## WRITE DATASET LOADER CODE HERE ##########
        # Just normalization for validation
        data_transforms = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
        }
        thispath = os.path.dirname(os.path.abspath(__file__))
        data_dir = os.path.join(thispath,"hymenoptera_data")        
        image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                                  data_transforms[x])
                          for x in ['train', 'val']}
        self.train_dataset = image_datasets['train']
        self.valid_dataset = image_datasets['val']
        ####################################################
        
    def get_train_dataset(self, validation=True):        
        if validation is True:
            return self.train_dataset, self.valid_dataset
        else:
            return self.train_dataset
    
    def get_test_dataset(self):
        return self.test_dataset

### test dataset

In [ ]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import os

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'datasets/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
script_params = {
    'epochs':25,
    'batch-size':4,
    'test-batch-size':8,
    'lr':0.01,
    'momentum':0.5,
    'seed':42,
    'log-interval':10,
    #'no-cuda':False,
    'nprocs':1,
    'loss':'cross_entropy',
    #'loss':'nll_loss',
    'optimizer':'SGD',
    'validation': True,
    'debug': True    
}

In [ ]:
estimator = Estimator(net_name=modulename,script_params=script_params)

In [ ]:
estimator.fit(dataset_loader=dataset_name)

<div class="alert alert-block alert-info">
<h1>Example 5(MNIST-.csv, .png)</h1>
</div>

In [ ]:
# USER-DEFINED - Feel free to change
modulename = "mnist-csv"
######### DO NOT CHANGE #########
net_filename = modulename + ".py"
net_filename = "./nets/" + net_filename
#################################

<div class="alert alert-block alert-warning">
    <p>Do not change code below at Line 1 to 3 </p>
    <p>Do not change class name <b>'Net'</b></p>
</div>

In [ ]:
%%writefile $net_filename
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

<div class="alert alert-block alert-info">
Feel free to change dataset name
</div>

In [ ]:
# USER-DEFINED
dataset_name = "mnist-images"
######### DO NOT CHANGE #########
dataset_filename = dataset_name + ".py"
dataset_filename = "./datasets/" + dataset_filename
#################################

In [ ]:
%%writefile $dataset_filename

from __future__ import print_function, division

import torch
import torchvision
import torchvision.transforms as transforms
import os

class DatasetLoader:    
    def __init__(self):
        ########## WRITE DATASET LOADER CODE HERE ##########
        # Just normalization for validation        
        mnist_transform=transforms.Compose([transforms.Grayscale(),
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.1307,), (0.3081,))])
        thispath = os.path.dirname(os.path.abspath(__file__))
        data_dir = os.path.join(thispath,"image-datasets","mnist","images")
        image_dataset = torchvision.datasets.ImageFolder(data_dir,mnist_transform)
        
        self.train_dataset = image_dataset
        ####################################################
        
    def get_train_dataset(self, validation=True):        
        if validation is True:
            return self.train_dataset, self.valid_dataset
        else:
            return self.train_dataset
    
    def get_test_dataset(self):
        return self.test_dataset

In [ ]:
script_params = {
    'epochs':5,
    'batch-size':32,
    'test-batch-size':64,
    'lr':0.01,
    'momentum':0.5,
    'seed':42,
    'log-interval':10,
    'no-cuda':True,
    'nprocs':1,
    #'loss':'cross_entropy',
    'loss':'nll_loss',
    'optimizer':'SGD',
    'debug': True
}

In [ ]:
estimator = Estimator(net_name=modulename,script_params=script_params)

In [ ]:
estimator.fit(dataset_loader=dataset_name)